# CS 445 Project Report: Nikhil-GPT, Machine Learning Language Model

*by Daniel Ives and Kenny Hotra, May 02, 2023*

## Introduction

In the past year, language models such as Chat-GPT and Bing Chat have quickly risen to prominence with millions of users using these models for learning, writing (excluding this report), and as a tool in their lives and jobs. With the rapid rise in this technology, we took it upon ourselves to answer the following question: Is it possible to construct our own language model tailored for answering ML-related questions by training it on lecture content in our machine learning course at CSU? To develop this machine learning language model which we have named Nikhil-GPT, we have practiced the many steps in the machine learning process from processing data to designing the model and training it with varying parameters to assessing the results of our model. To provide our model with the necessary textual data, we applied OpenAI’s Whisper model to transcribe the class’s recorded lectures on Echo360. Using this, we then trained a model using Andrej Karpathy’s nanoGPT, an easy-to-use repository for building generative pre-trained transformers. The result of these efforts was a model that allows users to enter prompts for the model to finish with its own generated text. With this model, we assessed it through qualitative comparisons to other language models such as GPT-2 and quantitative markers such as the model’s training loss and MFU.

While we were successful in creating a language model that can generate text based on the lecture data it trained on, we were unable to train the model such that it can successfully convey meaningful information, particularly in the context of answering a user’s prompts. Additionally, it appears that our model could be further improved in terms of its utilization of system resources. Through comparisons with other models including GPT-2, we gained a heightened sense of the sheer scale of training data, model size, and number of parameters necessary to create a language model that can create sensible and meaningful responses. Furthermore, this project provided us with an application of transformers that improved our understanding of the deep learning model and an introduction to many of the concepts and tools used in natural language processing and prominent chatbots and language models.

## Methods

### Data Collection - Sources and Tools

The first phase of this project involved obtaining the training data for our model. Since the goal was to develop a language model focused on the topic of machine learning, we decided that the lectures from our machine learning class would be an excellent source of data. The good news is that these lectures are available through Canvas’s Echo360 feature, so we had access to the audio recordings. We selected the first 19 lectures as our data which Daniel downloaded into mp3 files. With each lecture containing ~10,000 words on average, these lectures provided our model with hundreds of thousands of words to train on. The bad news is that our GPT model requires textual data, so we needed to transcribe the lectures. To address this issue, we decided to use Whisper, a speech recognition model developed by OpenAI, the creators of Chat-GPT.

In essence, Whisper is a transformer that uses cross-attention to predict the text corresponding to encoded audio. First, the input audio is converted into 30-second chunks that are transformed into a log-Mel spectrogram. This spectrogram is a logarithmic transformation on frequencies measured by the mel scale, a unit of frequencies not unlike hertz. Then, the spectrogram is encoded using 2 1D convolutional layers and Gaussian Error Linear Unit (GELU) as the activation function. Additionally, sinusoidal positional encoding is applied, resulting in many encoded blocks of data. These blocks are decoded using a cross-attention mechanism, and with learned positional encoding (position referring to the timing of audio frequencies), the model learns to predict which tokens will appear next. 

![Whisper Model Diagram](https://openaicom.imgix.net/d9c13138-366f-49d3-b8bd-cb3f5a973a5b/asr-summary-of-model-architecture-desktop.svg?fm=auto&auto=compress,format&fit=min&w=1919&h=1551)

<p style="text-align: center;">Figure 1. Diagram of how Whisper processes audio data into predictive token, Source: <a href=https://openai.com/research/whisper>https://openai.com/research/whisper</p>

Whisper offers 5 different model sizes (in order of size): tiny, base, small, medium, and large. The tiny model is the smallest with 39 million parameters and runs the fastest with a 1 GB VRAM requirement. The subsequent models are larger with each increase in model size running with x2-3 the parameters but taking x2-3 the time as well as extra VRAM.


|  Size  | Parameters | English-only model | Multilingual model | Required VRAM | Relative speed |
|:------:|:----------:|:------------------:|:------------------:|:-------------:|:--------------:|
|  tiny  |    39 M    |     `tiny.en`      |       `tiny`       |     ~1 GB     |      ~32x      |
|  base  |    74 M    |     `base.en`      |       `base`       |     ~1 GB     |      ~16x      |
| small  |   244 M    |     `small.en`     |      `small`       |     ~2 GB     |      ~6x       |
| medium |   769 M    |    `medium.en`     |      `medium`      |     ~5 GB     |      ~2x       |
| large  |   1550 M   |        N/A         |      `large`       |    ~10 GB     |       1x       |

<p style="text-align: center;">Table 1. Display of Whisper model sizes and their capabilities/requirements</p>

Deciding which model size produced the best transcriptions was an essential step for our project since a model can only be as good as the data put into it. Assessing a language model trained on nonsensical text in terms of grammar and sentence structure would prove to be a futile effort. Another reason for desiring a high-quality transcription was that we wanted to minimize spelling and grammar errors and hopefully avoid manual clean-up which would waste a good amount of time and work. Therefore, the best Whisper model for our purposes is the model that produces the best results in a timely manner.


### Data Collection - Transcription and Formatting

The following code cell was written and run by Kenny and was used to transcribe the lecture audio into text files. It requires the whisper package as well as the FFmpeg command line tool. Installation for all necessary libraries can be found here: https://github.com/openai/whisper. When the cell is run, it will ask the user which size model should be used and the name of the directory where the mp3 files are stored. Then, the code will load the English-only model (excluding the large model which does not have an English-only model) and transcribe each audio file, storing the transcript for each audio file in the “Transcripts” directory.

In [ ]:
import whisper
import os

def run_model(size, directory):
    
    try:
        model = whisper.load_model(size + ".en")
    except:
        model = whisper.load_model(size)
    audio_files = os.listdir(directory)
    
    for path in audio_files:
        if path.startswith("."):
            pass
        else:
            print("Transcribing", path, "...")
            result = model.transcribe(directory + "/" + path, verbose=False)
            file = open("Transcripts/" + size + "-" + path[:path.rindex(".")] + ".txt", "w")
            file.write(result["text"])
            file.close()
        
# Options
accepted_sizes = ["tiny", "base", "small", "medium", "large"]
good_to_go = True
try:
    size = input("Enter model size for Whisper to use (leave empty to default to \"base\"): ").lower()
    if size == "":
        size = "base"
    elif size not in accepted_sizes:
        raise Exception
    
    directory = input("Enter name of directory containing audio files (leave empty to default to \"Audio\"): ")
    if directory == "":
        directory = "Audio"
except:
    print("The given size is not accepted by Whisper.\nTry one of the following: ", end="")
    print(*accepted_sizes, sep = ", ")
    good_to_go = False
    
# Make directory for transcripts
try:
    os.mkdir("Transcripts")
except:
    pass # No need to make a directory if it already exists

# Only works if proper size and audio directory are selected
if good_to_go:
    print("Running", size, "Whisper model over", directory, "...")
    run_model(size, directory)

For this project, we transcribed the lectures using all model sizes except for the large model due to time and resource constraints. The decision to force models to transcribe to English was made after the medium model consistently misidentified and then failed to translate one of the lectures to Welsh. A possible explanation for this behavior is how a multilingual Whisper model analyzes the first 30 seconds of audio to predict the language of the remaining audio. Because of the background noise in the first 30 seconds of that recording, the model was unable to correctly identify its language as English. Overall, the results of the transcription went as expected. The smaller models ran at a fraction of the time as their larger counterparts but with harder-to-read and more incorrect text. A comparison can be made between the tiny, base, small, and medium outputs for the same excerpt regarding ReLU and activation functions.

Tiny Model: Much of the excerpt is structurally confusing and very hard to read. It’s also notable how this model’s excerpt is longer than the others and how it interprets ReLU as a name, “Ray Lou”.

> So what we're going to talk about today is Ray Lou, who knows what Ray Lou is. You know, what we're going to do is we're going to talk about. So we talked about activation functions, right activation functions are these non linear functions that we applied at the output of a hidden layer. And meaning that when we back propagate the error through the network, the weights in that in layer are effectively optimized with the assumption that a certain non linear function will be applied to that output.

Base Model: It's a bit more coherent, but it still struggles with the term “ReLU” and grammatical correctness.

> So, what we are going to talk about today is Ray Lou, who knows what Ray Lou is. Okay. So, we talked about activation functions right activation functions are these non linear functions that we applied to the output of a hidden layer, and meaning that when we back propagate the error through the network, the weights in that the network function layer are effectively optimized with the assumption that a certain non linear function will be applied to that output.

Small Model: Close to the actual lecture but with some spelling and grammar errors

> So what we are going to talk about today is re lu who knows what re lu is. Few of you. Okay. So, we talked about activation functions right activation functions are these non linear functions that we apply to the output of a hidden layer, and meaning that when we back propagate the error through the network, the weights in that hidden layer are effectively optimized with the assumption that a certain non linear function will be applied to that output.

Medium Model: Effectively 1 to 1 with the actual lecture

> So what we are going to talk about today is ReLU. Who knows what ReLU is? A few of you. So we talked about activation functions. Activation functions are these nonlinear functions that we apply to the output of a hidden layer. And meaning that when we back propagate the error through the network, the weights in that hidden layer are effectively optimized with the assumption that a certain nonlinear function will be applied to that output.

Comparing the transcriptions, we decided that the medium model produced the highest quality transcriptions, so we selected its outputs as the training data for our model. Not only did the medium model produce the most grammatically correct transcripts, but it was also the most successful in transcribing ML-specific vocabulary. Unfortunately, it was not practical for us to employ word error rate (WER) as a quantitative measure for assessing these transcriptions. Although WER would have been a useful measurement since it calculates the ratio of errors to the number of words spoken, constructing the manmade transcript to serve as ground truth for the generated transcripts would have taken too much time and was outside of the scope of this project. Despite the lack of quantitative reasons for selecting which transcripts to use, the high-quality output of the medium-sized Whisper model with some manual clean-up should provide a solid set of data for our language model as well as a useful reference when assessing our model.

Before sending the text data to the model, we made sure to perform some clean-up on it. Daniel corrected some of the errors in the transcript using spelling and grammar check tools in document editors and combined the transcripts into a single text file using this bash script:

```
#! /bin/bash

# given a directory of transcripts, combine them into a single file
# usage: combine-scripts.sh <directory> <output file>

# check for correct number of arguments
if [ $# -ne 2 ]; then
    echo "usage: combine-scripts.sh <directory> <output file>"
    exit 1
fi

# check that the directory exists
if [ ! -d $1 ]; then
    echo "error: $1 is not a directory"
    exit 1
fi

# check that the output file does not exist
if [ -f $2 ]; then
    echo "error: $2 already exists"
    exit 1
fi

# create the output file
touch $2

# loop through the files in the directory
for file in $1/*; do
    # check that the file is a regular file
    if [ -f $file ]; then
        # append the file to the output file
        cat $file >> $2
    fi
done

# exit with success
exit 0
```

### Training the Model

Once we collected all of the input data, we were ready to train. The first step in doing this is to take the text data collected in part one and turn it into tokens. Tokens are vector representations of text that preserve the meaning and relationship between words in the vector space. This happened using the `prepare.py` file from nanoGPT. This code takes our input as a text file and runs it through the tokenizer of choice with the Python library `tiktoken` (we chose the GPT2 tokens). Once it generates these tokens, it splits them into 90% training data and 10% validation data and stores them as files.

Now that we have our tokens, we’re ready to train. Using the `train.py` file,  we can specify the hyperparameters of the model and set it running. We started with a small model, copying the hyperparameters from Karpathy’s Shakespeare example in the Github repository. This ended with a not-so-great val loss of 5.4406 after 100 epochs. Using `sample.py`, we can use the model to get output. It manages to put words in mostly the correct order, but the content of it doesn’t make any sense. With no prompt, here’s what it gave us.

> from any 10, but I'm trying to be doing so if I want that gradient descent shape to cancel out as the exact or is basically just looking at this, because I'll do the amount of values. And I'm moving in the output tasks. So now I see 100 minutes, I'll have to take a lot of samples, and then I can do I can just take the sum of zero for that by one. So let's create the weight matrix of X and right so now we have all my velocity is just trying to deal with this validation or 1.

We also tried training a larger model. While the first one had 7.23M parameters, this new one had 19.17M. However, the model quickly overfits, ending with a train loss of 0.4823 and a val loss of 7.3903. We hypothesize that we don’t have enough training data to warrant a larger model. Instead, we can achieve a better result with fewer parameters and train it faster, anyway. Also, the weights file exceeds 100MB, meaning we couldn’t store it in Github. For both of these reasons, a smaller model is preferable.

Taking all that we learned, we wanted to train a final model for analysis. We chose to use the same parameters as the first time (from the nanoGPT readme), but also fine-tune.
This is the code run for the final model, as can be seen in `final-model-training.ipynb`. The following was written and run by Daniel Ives.

In [ ]:
! python3 train.py config/train_config.py --out_dir=out-nikhil-gpt-final_dirty --compile=False --eval_iters=200 --log_interval=1 --block_size=64 --batch_size=12 --n_layer=4 --n_head=4 --n_embd=128 --max_iters=2000 --lr_decay_iters=2000 --dropout=0.0

In [ ]:
! python3 train.py config/train_config.py --init_from=resume --out_dir=out-nikhil-gpt-final_dirty --compile=False --eval_iters=200 --log_interval=1 --block_size=64 --batch_size=12 --n_layer=4 --n_head=4 --n_embd=128 --max_iters=1000 --lr_decay_iters=1000 --dropout=0.2 --learning_rate=1e-4

After initial training with the first line, we also finetuned the model. This is simply more training with a smaller learning rate starting from the previous best weights of the initial training. The `train.py` file makes this quite simple, as you can just set the training mode to “resume” and decrease the learning rate.

Unfortunately, from a qualitative standpoint, this performed essentially the same as our first try. Before accepting the final results, we tried one last thing. We took our “medium” training data and passed it through a Google doc, auto-accepting all recommended grammar edits. The goal was to clean up some of the noise of the data, removing repeated words and poor grammar.
After initial training on our final “clean” model, we had a training loss of 3.5905 and a validation loss of 4.5120. Fine-tuning slightly improved this, we ended with a training loss of 3.5951 and a validation loss of 4.4653. Here is some sample output we got:

>Any questions about the policy. And you have your path so if I want it's a little bit. You want to handle a little bit data and an time to do this, you know, this for this. They're not going to be doing this is not going to be going to have a lot of different properties of the samples, and the value to be able to predict different weights that are actually always a single vector. So let's create the first one thing. And so that is the sum all my input values of these are done with this. So if I take more what I'm going to be the output of each of the target weight for all k and xt times k. So, then the sum now I have the softmax function. And then I can have a e to the derivative of course, what the term is the n is,, that I'm going to do is I can draw the sum for the output of all y.

Unfortunately, cleaning the data up in google docs didn’t seem to make any qualitative difference. Either way, the final model weights are in our repository, and can be downloaded here and used with the code in the *Results* section

### Developing Metrics for Assessment

#### Qualitative

Putting our model against GPT-2/3/4 isn’t exactly a fair comparison. Even GPT-2 was trained on ~40GB of text and 1.5 billion parameters. Compared to this, our model was only trained with 1.1MB of text and 7 million parameters. Nonetheless, 

Nikhil-GPT:

>matrix multiplication is *the gradient of the actual values. And we can see so if I want that gradient of the same thing that I want to see is now just looking at this, the test set of the data, I can see that it's the probability of the line is a 10% than zero. So I'm going to be the set of those, the value to be able to predict different weights, they actually get a single vector. So let's create the first one thing. And so that is the*

GPT-2:

>matrix multiplication is *a technique to increase the square of the vector. It can only work with very high levels of accuracy, but once the vector becomes high enough, then the precision at which it is applied disappears, and for such an approximation to be*

Based on subjective analysis, GPT-2 certainly wins as it successfully forms full sentences with correct grammar. Our model, by comparison, almost forms full sentences, but it doesn’t always get it right. For example, sentences like “So I’m going to be the set of those, the value to be able to predict different weights, they actually get a single vector” do not make any grammatical sense. Surprisingly, even GPT-2 isn’t very successful when it comes to the content of the text. Neither model forms cohesive text content.

We view this as a pretty big win considering the difference in size between Nikhil-GPT and GPT-2. If we had a comparable amount of data specific to machine learning, we think we could create a decently cohesive model.

#### Quantitative

To create objective metrics to assess our model and compare it to others, we settled on evaluating our model’s training loss on the training and validation sets as well as the MFU  (Model FLOPS Utilization). Assessing the model’s losses during training is useful because it allows us to determine whether the model is successfully generalizing and fitting the data. The difference in loss between training and validation sets can reveal whether our model is overfitting the data it is training on. To gain a greater understanding of how our model fits the training data, we assessed the training losses on both the “dirty” and “clean” data (that is, the data before and after manual clean-up). MFU is a measurement of the model’s efficiency in terms of resource utilization. It is represented as a percentage of the potential number of FLOPS (floating-point operations per second) that are used during training. Though this value can vary depending on the hardware that the model is training, it is a useful metric when comparing the efficiency of our model to other language models that have also adopted MFU as a performance metric.

Further reasoning for our selection of training loss and MFU as our quantitative metrics is that nanoGPT reports its losses and MFU during training. It estimates its losses on the training and validation sets once every few intervals, saving the model as a checkpoint if the validation loss has decreased since the previous estimate. For our purposes, we calculated the losses once every 10 iterations since estimating the losses takes a substantial amount of time relative to training during any given iteration. These reported values can be recorded for future analysis and discussion. Kenny was responsible for recording the graphs and tables that present these metrics in the *Results* section.


## Results

Below are the graphs plotting the model’s losses during training on both the dirty and clean data created using Matplotlib. Each plot is split into two phases based on the phases in which we trained our model. The first phase depicts the model’s first 400 iterations of training. The second phase depicts the model’s fine-tuning, which starts from the previous best checkpoint, where the validation loss was the lowest.

(Attachments: Loss plots on dirty data, Loss plots on clean data, available on GitHub)

For both sets of training data, the model quickly improves on its losses on both training and validation sets for the first 200 iterations. For the next 200 iterations, the training set losses decline steadily below 4 whereas the validation set loss becomes stagnant. During the second phase, both losses temporarily jump but then decrease steadily below their initial loss value. While both losses decline, the training set losses decline more quickly than the validation set losses. From these trends, we can conclude that the model begins to overfit the training data after a few hundred iterations. We can also conclude that although the architecture and parameters of the model can be modified such that validation loss is improved, the overfitting issue is not rectified.

Another notable observation is that the model’s validation loss was slightly lower (about 0.1 to 0.2 on average) on the dirty dataset compared to the clean dataset although the training losses are about the same between datasets. It was intuitive to predict that cleaning the transcripts to enforce grammatical correctness would improve the model’s performance, but doing so resulted in a smaller dataset with approximately a thousand fewer tokens. With this reduction in training data, the model may have been slightly more susceptible to overfitting.

Below is a table reporting the MFUs for our model when trained on different hardware compared to MFUs reported by other language models within Google Research’s PaLM report which is available under References.

| Model                          | # Params (Millions) | Hardware/Hostname              | Average MFU   | Source          |
|:------------------------------:|:-------------------:|:------------------------------:|:-------------:|:---------------:|
| Nikhil-GPT                     | 7                   | mercury.cs.colostate.edu       | 1.06%, 1.03%* | Model Training  |
| Nikhil-GPT                     | 7                   | wahoo.cs.colostate.edu         | 0.41%         | Model Training  |
| GPT-3                          | 175,000             | V100 accelerator chips         | 21.3%         | Google Research |
| Gopher                         | 280,000             | 4096 TPU v3 accelerator chips  | 32.5%         | Google Research |
| Megatron-Turing NLG            | 530,000             | 2240 A100 accelerator chips    | 30.2%         | Google Research |
| PaLM                           | 540,000             |  6144 TPU v4 accelerator chips | 46.2%         | Google Research |

<p style="text-align: center;">Table 2. Comparison of MFU across different models and hardware</p>
<p style="text-align: center;">*MFU varied between the 2 phases of training on this machine</p>

In terms of MFU, our model doesn’t hold a candle to the higher-end models that are used today. Of course, this is to be expected as we lack the resources that these larger organizations possess. Our model has less than a thousandth of the number of parameters compared to the models tested by Google’s research team and consequentially possesses a fraction of the MFU. We can also observe a relationship between the number of parameters in a model and the MFU of a model where. The more parameters a model has, the higher its MFU is. Another reason for the difference in MFU is the size of the training data employed by the models. Modern language models are trained on billions or trillions of training tokens whereas our model was trained on about a quarter-million.

Another observation regarding MFU is its fluctuation depending on the hardware used. For our model, the MFU varied from 0.41% to 1.06%. While there are ways to optimize our model and increase its MFU from further fine-tuning parameters to employing more powerful hardware, we would be more likely to see success either by implementing a larger model than nanoGPT or by including more training data for our model to work with.

### Examples and Demo

With no starting prompt, here are five samples that Nikhil-GPT output with a maximum of 100 tokens:

>Any questions about the policy. And you have your path so if I want it's a little bit. You want to handle a little bit data and an time to do this, you know, this for this. They're not going to be doing this is not going to be going to have a lot of different properties of the samples, and the value to be able to predict different weights that are actually always a single vector. So let's create the first one thing. And so that is the
---------------
>we don't want to do to say I'm going to take the forward pass more what I'm not going to be going to be taking the training training data. So this is going to be my predictions, then the second layer. I'm going to show the 10 by two. So I see how I'm going to do we'll get this, I'm going to see that we can see this, and then I can also use the last class two, and then I'm going to be
---------------
>I want to be able to do a bunch of study that if there's a three of things for each time, you can take the data kind of sort of say, which is the second set that you don't go to a lot of them that kind of a bunch of different of feature, you don't want to get to take like different types of training data which output or less complicated. And so you have to find a lot of training training data that you can use other things that of training
---------------
>Just yeah, yeah, all, you know, you know, let's say, you know, you use the assignment two. So if I go through this, yes. So I'm going to put I'm just getting no different outcomes. So I'm getting something like this, you know, kind of say, finally, what I'm going to do is I'm going to start trying to be, I'm going to say, I can try to do that to see what you can do
---------------
>Thanks, it's just not in a lecture. Yes. So you're not going to have a good job. You should be in time, but you got a couple of things like the project. So, that you're going to be able to look at this data. If you're going to use to keep in this data that. So you're going to keep in each time. And I'm just trying to do this functions as we'll show is, the value for this is that's
---------------

The following code cell downloads a zip file of our model weights and code to try out the model.

In [ ]:
! wget https://github.com/daives01/nikhil-GPT/raw/main/tryNikhilGPT.zip
! unzip tryNikhilGPT.zip

Assuming you downloaded and unzipped in the same directory, and have the python dependencies installed, you can run the following code cells to try out the model.

In [ ]:
# Feel free to adjust some parameters here
num_samples = 5 # number of samples to generate
max_new_tokens = 100 # maximum number of tokens to generate
temperature = 0.8 # 1.0 = no change, < 1.0 = less random, > 1.0 = more random, in predictions
top_k = 200 # retain only the top_k most likely tokens, clamp others to have 0 probability
seed = 1337

The following cell runs the model with no starting prompt.

In [ ]:
! python3 sample.py --out_dir=out-nikhil-gpt-final_clean --num_samples=$num_samples --max_new_tokens=$max_new_tokens --temperature=$temperature --top_k=$top_k --seed=$seed

The following cell runs the model with a starting prompt of your choosing.

In [ ]:
# Get user input
prompt = input("Enter a prompt: ")

# Generate text
! python3 sample.py --start="$prompt" --out_dir=out-nikhil-gpt-final_clean --num_samples=$num_samples --max_new_tokens=$max_new_tokens --temperature=$temperature --top_k=$top_k --seed=$seed

## Conclusions

This project was largely a success. Even though we didn’t create an amazing LLM, we learned a huge amount about how they work. We also managed to stay ahead of our schedule and adapted to challenges as they arose. For example, we originally planned to use BLEU scores as a quantitative metric, but the more we learned about them, the more we realized we wouldn’t be able to apply them to our project effectively. Instead, we used our model’s loss function and MFU compared to other big LLMs.

The main realization of this project is the sheer amount of data and compute power it takes to train transformers like GPT-3 and 4. We were hopeful that our model would perform decently given the relatively small scope of the task, but that turned out not to be the case at all. At the end of the day, the quality of the output depends entirely on the quality of the input, and our input wasn’t the best.

### Appendix

* The GitHub repository containing our work and struggles: https://github.com/daives01/nikhil-GPT

### References

* Nikhil Krishnaswamy, CS445
* Whisper: https://github.com/openai/whisper
* nanoGPT: https://github.com/karpathy/nanoGPT
* Alec Radford, Jong Wook Kim, Tao Xu, Greg Brockman, Christine McLeavey, Ilya Sutskever, Robust Speech Recognition via Large-Scale Weak Supervision (2022). https://arxiv.org/abs/2212.04356
* Google Research, et al., PaLM: Scaling Language Modeling with Pathways (2022). https://arxiv.org/abs/2204.02311 

In [1]:
import io
from nbformat import current
import glob
nbfile = glob.glob('Ives-Hotra-Project-Report.ipynb')
if len(nbfile) > 1:
    print('More than one ipynb file. Using the first one.  nbfile=', nbfile)
with io.open(nbfile[0], 'r', encoding='utf-8') as f:
    nb = current.read(f, 'json')
word_count = 0
for cell in nb.worksheets[0].cells:
    if cell.cell_type == "markdown":
        word_count += len(cell['source'].replace('#', '').lstrip().split(' '))
print('Word count for file', nbfile[0], 'is', word_count)

Word count for file Ives-Hotra-Project-Report.ipynb is 5375
